In [ ]:
csv_file="../results/result_jdrt_phone.csv"
subject_file="./sub_gmm_multi.csv"

In [ ]:
import os
import glob
import math
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import mean_squared_error

df = pd.read_csv(csv_file,sep=',',header=0)

In [ ]:
#pip install japanize-matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import japanize_matplotlib
%matplotlib inline
plt.rcParams["font.size"] = 25
plt.rcParams['figure.figsize'] = (13.0, 11.0)

In [ ]:
df.insert(1,'drt_no',df['ID'].astype(str).str[:3])
df['drt_no'] = df['drt_no'].astype(int)
df.insert(2,'person_int',df['ID'].astype(str).str[4])
df['person_int'] = df['person_int'].astype(int)
df.insert(3,'sex_int',df['ID'].astype(str).str[6])
df['sex_int'] = df['sex_int'].astype(int)
df.insert(4,'noise_int',df['ID'].astype(str).str[8])
df['noise_int'] = df['noise_int'].astype(int)
df.insert(5,'snr_int',df['ID'].astype(str).str[10])
df['snr_int'] = df['snr_int'].astype(int)

In [ ]:
drt_120_openjtalk=['z a i','s a i','d a k u','t a k u','g i j i','k i j i','g i N','k i N','z u i','s u i','g u u','k u u','z e i','s e i','d e b a','t e b a','z o u','s o o','g o j i','k o j i','m a N','b a N','n a i','d a i','m i s u','b i s u','m i r u','b i r u','m u r i','b u r i','m u sh i','b u sh i','m e N','b e N','n e r u','d e r u','m o N','b o N','n o r a','d o r a','h a sh i','k a sh i','h a t a','k a t a','sh i r i','ch i r i','h i r u','k i r u','s U k i','ts U k i','s u n a','ts u n a','h e N','k e N','h e r i','k e r i','h o sh i','k o sh i','h o r u','k o r u','j a m u','g a m u','ch a k u','k a k u','sh I k i','h i k i','ch i j i','k i j i','ch u u','k u u','j u N','g u N','sh e a','h e a','sh e r u','h e r u','j o o','g o o','sh o j i','h o j i','w a k u','r a k u','p a i','t a i','m i e','n i e','m i s u','n i s u','m u k u','n u k u','m u sh i','n u sh i','m a N','n e N','p e N','t e N','m o o','n o o','p o r o','t o r o','y a k u','w a k u','k a i','p a i','g i N','b i N','k i z a','p i z a','k u r o','p u r o','y u u','r u u','g e N','b a N','k e N','p e N','g o k i','b o k i','y o k a','r o k a']

In [ ]:
u = df['reference'].unique()
print(len(u))

In [ ]:
dic_sex={2:'male',1:'female'}
dic_snr={4:'-15',3:'-10',2:'0',1:'10',0:'clean'}
dic_noise={0:'non',1:'babble',2:'brown',3:'pseudo',4:'pink',5:'white'}
dic_person={1:'aoki',2:'kanno',3:'moriya',4:'nakamura',5:'minagawa',6:'miura',7:'nagata',8:'suzuki'}
dic_consonant={0:'voicing',1:'nasality',2:'sustention',3:'sibilation',4:'graveness',5:'compactness'}

In [ ]:
df.insert(8,'ref_front',df['reference'].str[0])
df.insert(9,'hyp_front',df['hypothesis'].str[0])
df.insert(10,'ref_theOther',"")
for index, data in df.iterrows():
    drtno_int=df.loc[index,'drt_no']
    if drtno_int%2==1:
        df.loc[index,'ref_theOther']=drt_120_openjtalk[drtno_int]
    elif drtno_int%2==0:
        df.loc[index,'ref_theOther']=drt_120_openjtalk[drtno_int-2]

df.insert(11,'ref_theOther_front',df['ref_theOther'].str[0])

In [ ]:
df.isnull().sum()

In [ ]:
#エラー・2択内・自由回答で出力を分類
for index, data in df.iterrows():
    ans_kana_front=df.loc[index,'hyp_front']
    if ans_kana_front==" ":
        df.loc[index,'txt_kana_free']="error"
    elif ans_kana_front==df.loc[index,'ref_front'] or ans_kana_front==df.loc[index,'ref_theOther_front']:
        df.loc[index,'txt_kana_free']=df.loc[index,'hyp_front']
    else :
        df.loc[index,'txt_kana_free']="free"

In [ ]:
#consonant
df.insert(8,'Consonant',"")
df.loc[df['drt_no']<=20,'Consonant']=0
df.loc[(df['drt_no']>20)&(df['drt_no']<=40),'Consonant']=1
df.loc[(df['drt_no']>40)&(df['drt_no']<=60),'Consonant']=2
df.loc[(df['drt_no']>60)&(df['drt_no']<=80),'Consonant']=3
df.loc[(df['drt_no']>80)&(df['drt_no']<=100),'Consonant']=4
df.loc[df['drt_no']>=101,'Consonant']=5
df['Consonant']=df['Consonant'].astype(int)

#word_pair
df.insert(9,'versus',"")
for index, data in df.iterrows():
    temp=df.loc[index,'drt_no'].astype(int)
    if temp%2==1:
        temp=temp+1
    df.loc[index,'versus']=temp//2
df['versus']=df['versus'].astype(int)

In [ ]:
#noise限定
df=df[df['noise_int']!=2]
df=df[df['noise_int']!=4]

In [ ]:
#crr:正(1),誤(0),自由回答(乱数),未認識(乱数)??
#0:誤答扱い, random:乱数, nan:ノーカウント
ERROR="random"
FREE="random"

import random
random.seed(0)

df2=df
df2['crr']=""
for index, data in df2.iterrows():
    samp=df.loc[index,'txt_kana_free']
    if samp== 'error':
        if ERROR=="0":
            df2.loc[index,'crr']=0
        elif ERROR=="random":
            df2.loc[index,'crr']=random.randint(0, 1)
        elif ERROR=="nan":
            df2.loc[index,'crr']=np.nan
    elif samp=='free':
        if FREE=="0":
            df2.loc[index,'crr']=0
        elif FREE=="random":
            df2.loc[index,'crr']=random.randint(0, 1)
        elif FREE=="nan":
            df2.loc[index,'crr']=np.nan
    else:
        df2.loc[index,'crr'] = np.where(df2.loc[index,'ref_front'] ==df2.loc[index,'txt_kana_free'], 1,0)
print(np.unique(df2['crr']))

In [ ]:
#null判定を除外 出力は前後
print(df.shape)
df2=df2[df2['crr'].notnull()]
print(df2.shape)

In [ ]:
#ノイズ別了解度算出
evaluation=1
model='CTC'
target='crr'

x1='Consonant'
y1="snr_int"
z1='noise_int'
col_name=['evaluation','model','noise','snr','consonant','crr']

x=df2[x1].value_counts().to_dict()
y=df2[y1].value_counts().to_dict()
z=df2[z1].value_counts().to_dict()

#result_df= pd.DataFrame()
test2_list=[]
for i in z.keys():
    if i==0: #snrとnoiseの整合性
        continue
    df_copy=df2[df2['noise_int'].isin([0,i])]
    for j in x.keys():
        for k in y.keys():
            test_list=[]
            A=((df_copy[x1]==j)&(df_copy[y1]==k)&(df_copy[target]==1)).sum()
            B=((df_copy[x1]==j)&(df_copy[y1]==k)&(df_copy[target]==0)).sum()
            C=((df_copy[x1]==j)&(df_copy[y1]==k)).sum()
            a=(A-B)/C*100
            test_list.append(evaluation)
            test_list.append(model)
            test_list.append(i)
            test_list.append(k)
            test_list.append(j)
            test_list.append(a)
            test2_list.append(test_list)

result_df=pd.DataFrame(test2_list)
result_df.columns=col_name
result_df.head(5)

In [ ]:
#客観評価を並び替え、データフレームXで定義
temp_X= result_df.sort_values(['noise','snr','consonant'])
temp=temp_X.reset_index(drop=True)
X=temp['crr']
print(X.describe())

In [ ]:
#主観評価CSVを並び替え、Y定義
df_sub=pd.read_csv(subject_file, encoding="utf-8")
temp_Y=df_sub[df_sub['model']==0]
temp_Y=temp_Y.sort_values(['noise','snr','consonant'])
temp=temp_Y.reset_index(drop=True)
Y=temp['crr']
print(Y.describe())

In [ ]:
#散布図、ｒ、rmse算出
fig = plt.figure()
plt.scatter(X, Y)
senkei=np.polyfit(X,Y,1)
func=np.poly1d(senkei)
F=func(X)
plt.plot(X, F, label='best-fit line')
a=np.corrcoef(X,Y)[0][1]
rmse = np.sqrt(mean_squared_error(Y, X))
plt.xlabel("Objective [%]")
plt.ylabel("Subjective [%]")
plt.xlim(0,100)
plt.ylim(0,100)
plt.plot([0, 100], [0, 100], ls="--", c=".3", label='equal line')
plt.grid(True)
plt.legend()
plt.title("wav2vec: r={:.2f}, rmse={:.2f}".format(a,rmse))

print("r=%f"% a)
print('rmse=%f'%rmse)

fig.savefig("./scatter.png")
plt.close('all')

In [ ]:
#子音特徴別結果（ノイズ平均）
pd.options.display.precision=2
target='crr'
x1="snr_int"
y1='Consonant'

x=df2[x1].value_counts().to_dict()
y=df2[y1].value_counts().to_dict()
gdf=pd.DataFrame(index=x.keys(),columns=y.keys())
for i in x.keys():
    for j in y.keys():
        a=((df2[(df2[y1]==j)&(df2[x1]==i)&(df2[target]==1)].count())-(df2[(df2[y1]==j)&(df2[x1]==i)&(df2[target]==0)].count())) / df2[(df2[x1]==i)&(df2[y1]==j)].count() *100
        gdf.at[i,j]=a.iat[0]
gdf_s=gdf.sort_index(ascending=False)
gdf_s=gdf_s.rename(index=dic_snr)
gdf_s=gdf_s.rename(columns=dic_consonant)
gdf_s.round(2)
gdf_s

In [ ]:
plt.figure()
gdf_s.plot(title='Correct response rate with wav2vec \nfor each attribute vs. SNR',xlabel='SNR[dB]',ylabel='correct respnse rate[%]',ylim=[0,100],linewidth=7, marker="o", markersize=15, grid=True)
plt.savefig('./consonant.png')
plt.close('all')

In [ ]:
plt.close('all')

In [ ]:
pd.options.display.precision=2
df3=gdf_s.mean(axis='columns')
#df3=pd.DataFrame(df3)
df3.columns=['ctc']
df3

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#単語対での調査
target='crr'
x1="snr_int"
y1='versus'

x=df2[x1].value_counts().to_dict()
y=df2[y1].value_counts().to_dict()
gdf=pd.DataFrame(index=x.keys(),columns=y.keys())
for i in x.keys():
    for j in y.keys():
        a=((df2[(df2[y1]==j)&(df2[x1]==i)&(df2[target]==1)].count())-(df2[(df2[y1]==j)&(df2[x1]==i)&(df2[target]==0)].count())) / df2[(df2[x1]==i)&(df2[y1]==j)].count() *100
        gdf.at[i,j]=a.iat[0]
gdf=gdf.sort_index(ascending=False)
gdf.index=['-15','-10','0','10','clean']
gdf.round(2)
gdf

In [ ]:
gdf = pd.concat([gdf,pd.DataFrame(gdf.mean(axis=0),columns=['mean']).T])
gdf.sort_values(by='mean',axis=1, ascending=True, inplace=True)
for pair_int in gdf:
    re_name=str(pair_int)+": "+drt_120_openjtalk[(2*pair_int)-2]+"-"+drt_120_openjtalk[(2*pair_int)-1]
    gdf.rename(columns={pair_int: re_name}, inplace=True)

gdf.loc['mean'].plot.barh(fontsize = 10)
plt.title('word-pair (ave.)',fontsize=15)
plt.xlabel('CRR[%]',fontsize=15)
plt.savefig('./word_pair.png')
plt.close('all')